In [1]:
from pyspark.sql import SparkSession

In [2]:
ss = SparkSession.builder.appName("kickstarter").getOrCreate()

In [3]:
df = ss.read.parquet("../data/kickstarter.parquet")

AnalysisException: 'Path does not exist: file:/home/grenciss/Documents/code/Al-Karid/data/kickstarter.parquet;'

Getting useful columns

In [ ]:
columns = df.columns
list(enumerate(df.columns))

In [ ]:
clf_columns = [columns[6],columns[7],columns[8],columns[9],columns[10],columns[11],columns[12]]

In [ ]:
df.select(clf_columns).show(5)

Fearuee engineering

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [ ]:
indexer_country = StringIndexer(inputCol=columns[6], outputCol=columns[6]+"_index").fit(df)
indexer_currency = StringIndexer(inputCol=columns[7], outputCol=columns[7]+"_index").fit(df)

In [ ]:
df = indexer_country.transform(df)
df = indexer_currency.transform(df)

In [ ]:
encoder_country = OneHotEncoder(inputCol=columns[6]+"_index", outputCol=columns[6]+"_vector")
encoder_currency = OneHotEncoder(inputCol=columns[7]+"_index", outputCol=columns[7]+"_vector")

In [ ]:
df = encoder_country.transform(df)
df = encoder_currency.transform(df)

In [ ]:
f_columns = [columns[6]+"_vector", columns[7]+"_vector", columns[11], columns[12]]
df.select(f_columns).show(5)

Assembling the vectors

In [ ]:
df_assembler = VectorAssembler(inputCols=f_columns, outputCol="features")
df = df_assembler.transform(df)

In [ ]:
df.select("features", columns[5]).show(5, False)